In [1]:
import numpy as np

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)} is not supported")
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func) -> None:
        self.creator = func
        return None
        
    def backward(self) -> None:
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                x.grad = gx
                
                if x.creator is not None:
                    funcs.append(x.creator)
        
        return None    


In [6]:
def as_array(x) -> np.ndarray:
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
        
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):
        raise NotImplementedError()
        

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy

    
def add(x0, x1):
    return Add()(x0, x1)


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx
    

def square(x):
    return Square()(x)

In [4]:
x = Variable(np.array(3.0))
y = add(x, x)
print("y", y.data)

y.backward()
print("x.grad", x.grad)

if x.grad != 2:
    raise ArithmeticError("x.grad should be 2")

y 6.0
x.grad 1.0


ArithmeticError: x.grad should be 2

In [5]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)} is not supported")
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func) -> None:
        self.creator = func
        return None
        
    def backward(self) -> None:
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx  # can't use += operator
                
                if x.creator is not None:
                    funcs.append(x.creator)
        
        return None    


In [7]:
x = Variable(np.array(3.0))
y = add(x, x)
print("y", y.data)

y.backward()
print("x.grad", x.grad)

if x.grad != 2:
    raise ArithmeticError("x.grad should be 2")

y 6.0
x.grad 2.0


In [9]:
x = Variable(np.array(3.0))
y = add(add(x, x), x)
print("y:", y.data)

y.backward()
print("x.grad:", x.grad)

if x.grad != 3:
    raise ArithmeticError("x.grad should be 3")

y: 9.0
x.grad: 3.0


# clear gradient

In [10]:
# First calculation
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

if x.grad != 2:
    raise ArithmeticError("x.grad should be 2")

# Second calculation
y = add((add(x, x)), x)
y.backward()
print(x.grad)

if x.grad != 3:
    raise ArithmeticError("x.grad should be 3")

2.0
5.0


ArithmeticError: x.grad should be 3

In [11]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)} is not supported")
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func) -> None:
        self.creator = func
        return None
        
    def backward(self) -> None:
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    funcs.append(x.creator)
        
        return None
    
    def cleargrad(self):
        self.grad = None


In [12]:
# First trial
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

if x.grad != 2:
    raise ArithmeticError("x.grad should be 2")

# Second trial
x.cleargrad()  # before second calculation, clear x.grad
y = add((add(x, x)), x)
y.backward()
print(x.grad)

if x.grad != 3:
    raise ArithmeticError("x.grad should be 3")

2.0
3.0
